# importing library

In [1]:
from __future__ import print_function
import keras
from keras.models import Model
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import plot_model
from PIL import Image , ImageOps
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
import random
import os
import math
import numpy as np
import keras.backend as K

Using TensorFlow backend.


# Creating train and test set

In [2]:
# Creation of Test data

input_folder = './ResizedImg'

x_train =[]
y_train =[]
x_test  =[]
y_test  =[]

percent = 0.6

for classes in os.listdir(input_folder):
    print(classes,' ',end = '')
    imagePath = input_folder+'/'+classes
    class_data = []
    for images in os.listdir(imagePath):
        try:
            im = Image.open(imagePath+'/'+images)
            imgMat = np.array(im)
            if imgMat.shape == (224,224,3): 
                class_data.append(imgMat)
        except:
            print('LoL')
    print(len(class_data))
    sizeofclass = len(class_data)
    
    num_of_Train_exp = math.floor(sizeofclass*(percent))
    
    X_train = class_data[0:num_of_Train_exp]
    X_test = class_data[num_of_Train_exp:]
    
    Y_train = [int(classes)-1]*len(X_train)
    Y_test = [int(classes)-1]*len(X_test)
    
    x_train = x_train + X_train
    y_train = y_train + Y_train
    x_test = x_test + X_test
    y_test = y_test + Y_test

del X_train
del Y_train
del X_test
del Y_test



1  177
10  323
11  466
12  1741
13  206
14  568
15  248
16  279
17  207
18  714
19  LoL
273
2  573
20  374
3  653
4  373
5  383
6  231
7  381
8  608
9  834


In [5]:
print(len(x_train))
print(len(y_train))

print(len(x_test))
print(len(y_test))


5755
5755
3857
3857


In [6]:
a = list(zip(x_train,y_train))
random.shuffle(a)
x_train , y_train = zip(*a)

a = list(zip(x_test,y_test))
random.shuffle(a)
x_test , y_test = zip(*a)
del a

In [8]:
x_train = np.array(list(x_train), dtype=np.float32)
y_train = np.array(list(y_train), dtype=np.float32)
x_test  = np.array(list(x_test), dtype=np.float32)
y_test  = np.array(list(y_test), dtype=np.float32)
y_train = keras.utils.np_utils.to_categorical(y_train, 20)
y_test = keras.utils.np_utils.to_categorical(y_test, 20)

# Normalization

In [9]:
x_train /= 255;
x_test /= 255;

# Saving train and test set in a file

In [10]:
np.save('x_train.npy',x_train)
np.save('y_train.npy',y_train)
np.save('x_test.npy',x_test)
np.save('y_test.npy',y_test)

# Loading Train and Test Set

In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test  = np.load('x_test.npy')
y_test  = np.load('y_test.npy')

# CNN Architecture

In [3]:
# config
batch_size = 50
num_classes = 20
epochs = 3

# Model 1

In [ ]:
# input image dimensions
img_rows, img_cols ,channel = 224, 224 , 3


# convert class vectors to binary class matrices

K.clear_session()
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(3,3),strides=1,padding='same',activation='relu',input_shape=(224,224,3)))
# model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Conv2D(filters=20, kernel_size=(3,3),strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Conv2D(filters=9, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(5,5), strides=5, padding="same"))
model.add(Flatten())
model.add(Dense(1089,activation='relu'))
model.add(Dense(400,activation='relu'))
model.add(Dense(20,activation='softmax'))

model.summary()

# model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

# model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])


# Model 2

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

#base_model = VGG16(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(400, activation='relu')(x)
# and a logistic layer -- 
predictions = Dense(20, activation='softmax')(x)

m = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

for layer in m.layers[:249]:
    layer.trainable = False
for layer in m.layers[249:]:
    layer.trainable = True
    
m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(m.summary())

m.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = m.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

Train on 5755 samples, validate on 3857 samples
Epoch 1/2
 256/5755 [>.............................] - ETA: 17:09 - loss: 3.0615 - acc: 0.1602

# Model 3

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(400, activation='relu')(x)
# and a logistic layer -- 
predictions = Dense(20, activation='softmax')(x)

m = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

for layer in m.layers[:249]:
    layer.trainable = False
for layer in m.layers[249:]:
    layer.trainable = True
    
m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(m.summary())

m.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = m.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Model4

In [ ]:
# input image dimensions
img_rows, img_cols ,channel = 224, 224 , 3


# convert class vectors to binary class matrices

K.clear_session()
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(3,3),strides=1,padding='same',activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Conv2D(filters=64, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Conv2D(filters=256, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
#model.add(Dropout(0.2,noise_shape=None, seed=None))
#keras.layers.Dropout(rate, noise_shape=None, seed=None)
model.add(Dense(2000,activation='relu'))
model.add(Dense(20,activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 6)       168       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      1760      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 128)       73856     
__________

# Model 5

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(224,224,3))
# base_model = VGG16(weights='imagenet', include_top=False)
base_model.summary()
# x_test = keras.utils.np_utils.to_categorical(x_test,20)
# y_test = keras.utils.np_utils.to_categorical(y_test,20)
# # add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(400, activation='relu')(x)
# and a logistic layer -- 
predictions = Dense(20, activation='softmax')(x)

m = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

for layer in m.layers[:47]:
    layer.trainable = False
for layer in m.layers[48:]:
    layer.trainable = True
    
m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(m.summary())

m.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = m.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\LASII\Anaconda3\envs\AdvanceCV\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Train on 5755 samples, validate on 3857 samples
Epoch 1/3
 400/5755 [=>............................] - ETA: 1:01:17 - loss: 3.0147 - acc: 0.2800

# Garbage Code

In [ ]:
# X_TEST_DATA = []
# Y_TEST_DATA = []

# input_folder_test = './ResizedImg/test'
# for classes in os.listdir(input_folder):
#     imagePath = input_folder+'/'+classes
#     for images in os.listdir(imagePath):
#         im = Image.open(imagePath+'/'+images)
#         imgMat = np.array(im)
#         X_TEST_DATA.append(imgMat)
#         Y_TEST_DATA.append(classes)

# X_TRAIN_DATA = []
# Y_TRAIN_DATA = []

# input_folder_test = './ResizedImg/train'
# for classes in os.listdir(input_folder):
#     imagePath = input_folder+'/'+classes
#     for images in os.listdir(imagePath):
#         im = Image.open(imagePath+'/'+images)
#         imgMat = np.array(im)
#         X_TRAIN_DATA.append(imgMat)
#         Y_TRAIN_DATA.append(classes)

# x_train_F = [item for sublist in x_train for item in sublist]
# y_train_F = [item for sublist in y_train for item in sublist]
# x_test_F  = [item for sublist in x_test for item in sublist]
# y_test_F  = [item for sublist in y_test for item in sublist]

# x_train_w = []
# y_train_w = []
# x_test_w = []
# y_test_w = []

# for img in x_train:
#     try:
#         if img.shape == (224,224,3):
#             x_train_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# for img in y_train:
#     try:
#         if img.shape == (224,224,3):
#             y_train_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# for img in x_test:
#     try:
#         if img.shape == (224,224,3):
#             x_test_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# for img in y_test:
#     try:
#         if img.shape == (224,224,3):
#             y_test_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# x_train = np.array(x_train_w, dtype=np.float32)
# y_train = np.array(y_train_w, dtype=np.float32)
# x_test  = np.array(x_test_w, dtype=np.float32)
# y_test  = np.array(y_test_w, dtype=np.float32)

# img = Image.fromarray(np.array(X_train[0]))
# img


# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))




# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train[0])
# print(len(x_train[0]))
# im = Image.open(x_train[0])
# im.show()
# print('before changing')
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')


# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
#     print("hola")
# else:
#     print("hola8")
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# print(x_train[0])
# print(len(x_train[0][0][0]))
# print(x_train[0].shape)
# print(np.zeros((3,3,1)))

# im1 = Image.open("b.jpeg").convert('LA')
# im1 = Image.open("b.jpeg")
# im2=ImageOps.grayscale(im1)
# Arr1 = np.array(im1)
# Arr2 = np.array(im2)
# print('shape of image is',Arr.shape,' value are ',Arr[0][0])
# Arr.show()
# img = Image.fromarray(Arr)
# img.show()    

# print(x_train[0].type)


In [63]:
np.random.random((10,2))

array([[0.67423311, 0.0430721 ],
       [0.57733056, 0.40607748],
       [0.16510067, 0.72085671],
       [0.56879062, 0.70950606],
       [0.6532192 , 0.54556054],
       [0.88227646, 0.6845853 ],
       [0.45780046, 0.50631469],
       [0.34744537, 0.40783336],
       [0.21499988, 0.44303972],
       [0.0415734 , 0.76062591]])

In [66]:
 keras.utils.np_utils.to_categorical?